**安装并导入相关包**

In [ ]:
!pip install datasets
!pip install trl

**自监督模式微调方法：输入一段文字，对每个进行预测。**

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from trl import SFTConfig, SFTTrainer

In [ ]:
dataset = load_dataset('imdb')

In [ ]:
small_train_dataset = dataset["train"].select(range(10))
small_eval_dataset = dataset["test"].select(range(10))

In [ ]:
small_train_dataset[0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

training_args = SFTConfig(
    eval_strategy="epoch",
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
    logging_steps=1,
    report_to='none'
)
trainer = SFTTrainer(
    model,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    args=training_args,
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.284200,3.496737
2,3.003500,3.503437


Epoch,Training Loss,Validation Loss
1,3.284200,3.496737
2,3.003500,3.503437
3,2.527000,3.512385


TrainOutput(global_step=6, training_loss=3.0260393619537354, metrics={'train_runtime': 286.4242, 'train_samples_per_second': 0.105, 'train_steps_per_second': 0.021, 'total_flos': 25682284216320.0, 'train_loss': 3.0260393619537354, 'epoch': 3.0})

**问答模式微调方法：question: text1, answer: text2, 模型对text2的每个字进行预测。**



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
dataset = load_dataset("sahil2801/CodeAlpaca-20k")

In [ ]:
dataset1 = dataset['train'].train_test_split(test_size=0.3)
small_train_dataset = dataset1['train'].select(range(20))
small_val_dataset = dataset1['test'].select(range(20))

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

training_args = SFTConfig(
    eval_strategy="epoch",
    output_dir="/tmp",
    logging_steps=1,
    report_to='none'
)

trainer = SFTTrainer(
    model,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)
trainer.train()

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,1.992300,1.857667
2,1.616800,1.833588
3,1.192900,1.821879


TrainOutput(global_step=9, training_loss=1.4840684864256117, metrics={'train_runtime': 475.9159, 'train_samples_per_second': 0.126, 'train_steps_per_second': 0.019, 'total_flos': 62489311248384.0, 'train_loss': 1.4840684864256117, 'epoch': 3.0})

**对话模式语言模型：输入human: text1, assistant: text2, human: text3, assistant: text4, human: text5, assistant: text6, ..., 模型对assistant对应的text的每个字进行预测。**

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, DatasetDict
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import re
import random
from multiprocessing import cpu_count

In [3]:
from huggingface_hub import login
login(token="here is your token")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')

In [ ]:
# 加载数据
dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
dataset = DatasetDict({
    "train": dataset["train_sft"],
    "test": dataset["test_sft"]
})

In [25]:
dataset['train'][0]

{'prompt': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
 'prompt_id': 'f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57',
 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of 

In [ ]:
# 处理数据
chat_template = """
{% for message in messages %}\n{%
if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n
{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{%
elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{%
endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}
"""
tokenizer.chat_template = chat_template

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return example

dataset1 = dataset.map(
            apply_chat_template,
            num_proc=cpu_count(),
            fn_kwargs={"tokenizer": tokenizer},
            remove_columns=list(dataset["train"].features),
            desc="Applying chat template",
      )


In [14]:
small_train_dataset = dataset1["train"].select(range(10))
small_test_dataset = dataset1["test"].select(range(10))

In [26]:
dataset1['train']

Dataset({
    features: ['text'],
    num_rows: 207865
})

In [27]:
dataset1['train'][0]

{'text': "\n<|system|>\n<|eot_id|>\n<|user|>\nThese instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?<|eot_id|>\n\n<|assistant|>\nThis feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.<|eot_id|>\n<|user|>\nCan you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?<|eot_id|>\n\n<|assistant|>\nSure, here are 

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [28]:
# 训练数据
training_args = SFTConfig(
    eval_strategy="epoch",
    output_dir="/tmp",
    overwrite_output_dir=True,
    dataset_text_field="text",
    packing=True,
    logging_steps=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    report_to="none"
)


trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_test_dataset,
        tokenizer=tokenizer,
)

trainer.train()

<ipython-input-28-8d4d459e46c1>:15: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Tokenizing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.010100,2.226725
2,0.056200,2.049299
3,0.030800,2.026891


TrainOutput(global_step=45, training_loss=0.052235146787845424, metrics={'train_runtime': 84.1997, 'train_samples_per_second': 0.534, 'train_steps_per_second': 0.534, 'total_flos': 261155436908544.0, 'train_loss': 0.052235146787845424})

SFT之前推理

In [22]:
from huggingface_hub import login
login(token="here is your token")

In [29]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a frendly teacher, who always reply in a gentle style!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Hello dear! I'm so glad you asked. I'm a friendly teacher, and I'm here to help you with any questions or topics you'd like to explore. I'm a warm and welcoming person, and I want to make sure you feel comfortable and supported as we chat. I'm here to listen, to offer guidance and advice, and to help you grow and learn in a safe and nurturing environment. What's on your mind today, my dear?"}


In [30]:
# sft之后推理
model_id = "/tmp/checkpoint-45/"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a frendly teacher, who always reply in a gentle style!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Device set to use cuda:0


{'role': 'assistant', 'content': "I am a student in your class, and I am grateful to have such a great teacher like you. Your classes are always so informative and fun to learn from. Can you tell me more about the topic we're currently studying in class? I'm curious to know more about it. Additionally, can you give me some tips on how to prepare for the upcoming quizzes and tests? I want to make sure I'm well-prepared for whatever comes next. Can you provide me with some extra help and support? I appreciate your efforts, and I'm looking forward to the next class! Can you give me some examples of real-world applications of the concepts we're learning in class? I'm curious to know more about how these ideas and principles apply to the world outside the classroom walls. Can you give me some examples of innovative solutions to real-world problems? I'm curious to know more about how students can apply theoretical concepts to practical, real-world problems. Can you give me some tips on how t